Import neccessary packages

In [1]:
import numpy as np
import pickle
import cv2
from os import listdir
from sklearn.preprocessing import LabelBinarizer   #for converting into binary form
from keras.models import Sequential                #for Sequential model generation
from keras.layers.normalization import BatchNormalization   #for normalizing
from keras.layers.convolutional import Conv2D              #for convolution layer
from keras.layers.convolutional import MaxPooling2D        #for max pooling layer
from keras.layers.core import Activation, Flatten, Dropout, Dense  #for dense , flatten,activation
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator   #for generating image data
from keras.optimizers import Adam    #for optimizing our model
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt     #for plotting graph
!pip install setproctitle
!pip install redis

Using TensorFlow backend.


  Retrying (Retry(total=4, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<pip._vendor.urllib3.connection.VerifiedHTTPSConnection object at 0x7f599143a6d8>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution',)': /simple/setproctitle/
  Retrying (Retry(total=3, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<pip._vendor.urllib3.connection.VerifiedHTTPSConnection object at 0x7f599143a5f8>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution',)': /simple/setproctitle/
  Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<pip._vendor.urllib3.connection.VerifiedHTTPSConnection object at 0x7f599143a668>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution',)': /simple/setproctitle/
  Retrying (Retry(total=1

In [2]:
import ray 
ray.init(ignore_reinit_error=True)

Process STDOUT and STDERR is being redirected to /tmp/raylogs/.
Waiting for redis server at 127.0.0.1:40291 to respond...
Waiting for redis server at 127.0.0.1:60241 to respond...
Starting the Plasma object store with 4.00 GB memory.
Starting local scheduler with the following resources: {'CPU': 2, 'GPU': 1}.

View the web UI at http://localhost:8889/notebooks/ray_ui13084.ipynb?token=50cf036b58b574af5e56e5f1495e9198a61b1f27418495ca



{'node_ip_address': '172.19.2.2',
 'redis_address': '172.19.2.2:40291',
 'object_store_addresses': [ObjectStoreAddress(name='/tmp/plasma_store42002106', manager_name='/tmp/plasma_manager90465828', manager_port=13602)],
 'local_scheduler_socket_names': ['/tmp/scheduler66729903'],
 'raylet_socket_names': [],
 'webui_url': 'http://localhost:8889/notebooks/ray_ui13084.ipynb?token=50cf036b58b574af5e56e5f1495e9198a61b1f27418495ca'}

In [3]:
import time
print("Timer started\n") 
print("-----------------------------------------------------------------------------------------") 
start = time.time()

Timer started

-----------------------------------------------------------------------------------------


In [4]:
EPOCHS = 5  #for specifying number of epochs
INIT_LR = 1e-3
BS = 32
default_image_size = tuple((256, 256))
image_size = 0
directory_root = '../input/plantvillage/'
width=256
height=256
depth=3

Function to convert images to array

In [5]:
@ray.remote(num_gpus=1)
def train_fit4():
    def convert_image_to_array(image_dir): # for skimming through the folder 
        try:
            image = cv2.imread(image_dir) 
            if image is not None :
                image = cv2.resize(image, default_image_size)
                return img_to_array(image) 
            else :
                return np.array([]) 
        except Exception as e: 
            print(f"Error : {e}")
            return None

In [6]:
def convert_image_to_array(image_dir): # for skimming through the folder 
    try:
        image = cv2.imread(image_dir) 
        if image is not None :
            image = cv2.resize(image, default_image_size)
            return img_to_array(image) 
        else :
            return np.array([]) 
    except Exception as e: 
        print(f"Error : {e}")
        return None

Fetch images from directory

In [7]:
image_list, label_list = [], [] 
try:
    print("[INFO] Loading images ...") 
    root_dir = listdir(directory_root) 
    for directory in root_dir :
        # remove .DS_Store from list 
        if directory == ".DS_Store" :     #for removing metadata
            root_dir.remove(directory)
            # for removing metadata
    for plant_folder in root_dir :
        plant_disease_folder_list = listdir(f"{directory_root}/{plant_folder}")
        for disease_folder in plant_disease_folder_list : 
            # remove .DS_Store from list
            if disease_folder == ".DS_Store" :
                plant_disease_folder_list.remove(disease_folder)
                
        for plant_disease_folder in plant_disease_folder_list: 
            print(f"[INFO] Processing {plant_disease_folder} ...") 
            plant_disease_image_list = listdir(f"{directory_root}/{plant_folder}/{plant_disease_folder}/")
            for single_plant_disease_image in plant_disease_image_list : 
                if single_plant_disease_image == ".DS_Store" :
                    plant_disease_image_list.remove(single_plant_disease_image)
           
            for image in plant_disease_image_list[:200]: 
                image_directory = f"{directory_root}/{plant_folder}/{plant_disease_folder}/{image}"
                if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True:
                    image_list.append(convert_image_to_array(image_directory))
                    label_list.append(plant_disease_folder) 
                    print("[INFO] Image loading completed")
except Exception as e: 
    print(f"Error : {e}")


[INFO] Loading images ...
[INFO] Processing Tomato_Late_blight ...
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[IN

[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] I

[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Processing Tomato_Spider_mites_Two_spotted_spider_mite ...
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INF

[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] I

[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] I

...

[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Processing Tomato_Early_blight ...
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading complet

[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] I

[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Processing Tomato_Bacterial_spot ...
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading compl

[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] I

[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] I

Get Size of Processed Image

In [8]:
image_size = len(image_list)
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list) 
pickle.dump(label_binarizer,open('label_transform.pkl', 'wb')) 
n_classes = len(label_binarizer.classes_) # for generating classes


Transform Image Labels uisng [Scikit Learn](http://scikit-learn.org/)'s LabelBinarizer

Print the classes

In [9]:
print(label_binarizer.classes_)

['Pepper__bell___Bacterial_spot' 'Pepper__bell___healthy'
 'Potato___Early_blight' 'Potato___Late_blight' 'Potato___healthy'
 'Tomato_Bacterial_spot' 'Tomato_Early_blight' 'Tomato_Late_blight'
 'Tomato_Leaf_Mold' 'Tomato_Septoria_leaf_spot'
 'Tomato_Spider_mites_Two_spotted_spider_mite' 'Tomato__Target_Spot'
 'Tomato__Tomato_YellowLeaf__Curl_Virus' 'Tomato__Tomato_mosaic_virus'
 'Tomato_healthy']


In [10]:
np_image_list = np.array(image_list, dtype=np.float16) / 225.0

In [11]:
print("[INFO] Spliting data to train, test")
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.2, random_state = 42) 

[INFO] Spliting data to train, test


In [12]:
aug = ImageDataGenerator(
    rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, 
    zoom_range=0.2,horizontal_flip=True, 
    fill_mode="nearest")

In [13]:
def create_and_fit_model():
    width=256
    height=256
    depth=3
    EPOCHS=2
model = Sequential()
inputShape = (height, width, depth)
chanDim = -1
if K.image_data_format() == "channels_first":
    inputShape = (depth, height, width)
    chanDim = 1
model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(n_classes))
model.add(Activation("softmax"))

Model Summary

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 256, 256, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 256, 256, 32)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 256, 256, 32)      128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 85, 85, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 85, 85, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 85, 85, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 85, 85, 64)        0         
__________

In [15]:
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS) # optimizing the model 
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"]) 
model.fit_generator(
    aug.flow(x_train, y_train, batch_size=BS), 
    validation_data=(x_test, y_test), 
    steps_per_epoch=len(x_train) //BS, 
    epochs=EPOCHS, verbose=1
)
print("[INFO] Calculating model accuracy") 
scores = model.evaluate(x_test, y_test) 
print(f"Test Accuracy: {scores[1]*100}")

create_and_fit_model()
output = train_fit4.remote() # for calling the function to be parallelized using ray
ray.get(output)

print("--------------------------------------------------------------------------------------------") 
end = time.time()
print("Time elapsed is {}".format(end-start)) # print time taken

Epoch 1/5
73/73 [==============================] - 46s 628ms/step - loss: 0.2045 - acc: 0.9343 - val_loss: 0.5692 - val_acc: 0.9120
Epoch 2/5
73/73 [==============================] - 35s 479ms/step - loss: 0.1643 - acc: 0.9451 - val_loss: 0.5000 - val_acc: 0.9170
Epoch 3/5
73/73 [==============================] - 35s 479ms/step - loss: 0.1455 - acc: 0.9504 - val_loss: 0.7166 - val_acc: 0.9008
Epoch 4/5
73/73 [==============================] - 36s 494ms/step - loss: 0.1166 - acc: 0.9600 - val_loss: 0.3423 - val_acc: 0.9260
Epoch 5/5
73/73 [==============================] - 35s 474ms/step - loss: 0.1091 - acc: 0.9613 - val_loss: 0.3459 - val_acc: 0.9277
[INFO] Calculating model accuracy
591/591 [==============================] - 2s 3ms/step
Test Accuracy: 92.76931751803093
--------------------------------------------------------------------------------------------
Time elapsed is 300.3998727798462
